# 만개의 레시피 csv파일에서 재료부분만 추출하기 위한 노트북
- 양념 리스트 또한 추출

In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
columns = ['레시피일련번호','데이터','재료만']
data = {'레시피일련번호' :  [12345, 23456, 34567, 45678],
        '데이터' : [[['당근','1개'],['양파', '2개'], ['밀가루', '500g'], ['참기름', '1스푼'],[]],
                    [['양배추', '1개'], ['고추장','2스푼'],['식용유', '1컵']],
                    [['당근','1개'],['밥','2공기'],['찰밥','2공기'],['드라이 이스트', '2개'],['소금','약간'],['올리브유','적당량']],
                    [['고추장','2스푼'],['식용유', '1컵']]],
        '재료만' : [['당근','양파','밀가루'],
                    ['양배추'],
                    ['인스턴스 드라이 이스트','밥','찰밥','당근 1/2개'],
                 []]
       }
test_df = pd.DataFrame(columns=columns, data=data)
test_df

,레시피일련번호,데이터,재료만
0,12345,"[[당근, 1개], [양파, 2개], [밀가루, 500g], [참기름, 1스푼], []]","[당근, 양파, 밀가루]"
1,23456,"[[양배추, 1개], [고추장, 2스푼], [식용유, 1컵]]",[양배추]
2,34567,"[[당근, 1개], [밥, 2공기], [찰밥, 2공기], [드라이 이스트, 2개],...","[인스턴스 드라이 이스트, 밥, 찰밥, 당근 1/2개]"
3,45678,"[[고추장, 2스푼], [식용유, 1컵]]",[]


In [3]:
for i in range(len(test_df)):
    lst = []
    for data in test_df['데이터'][i]:
        if len(data) == 0:
            pass
        else:
            if len(test_df['재료만'][i]) == 0:
                    lst.append(data)
            for word in test_df['재료만'][i]:
                if data[0] in word:
                    lst.append(data)
    print(lst)

[['당근', '1개'], ['양파', '2개'], ['밀가루', '500g']]
[['양배추', '1개']]
[['당근', '1개'], ['밥', '2공기'], ['밥', '2공기'], ['찰밥', '2공기'], ['드라이 이스트', '2개']]
[['고추장', '2스푼'], ['식용유', '1컵']]


In [4]:
try:
    path = 'C:\\Users\\User\\Desktop\\AIB_13\\CP1\\data\\만개의 레시피.csv'
except:
    path = '/content/drive/MyDrive/CP1/만개의 레시피.csv'
df = pd.read_csv(path, encoding='cp949')
df = df.dropna()
df = df.reset_index(drop=True)
df.shape


(114512, 18)

In [5]:
data = df['CKG_MTRL_CN'].copy()
data = data.dropna()
data

0         [재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...
1                  [재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간
2         [재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...
3         [재료] 파스타면 [양념] 토마토 1개| 토마토 페이스트 3T| 양파 1/2개| 다...
4         [재료] 건표고버섯 9개| 오이 1/2개| 당근 1/2개| 양파 1/2개| 사과 1...
                                ...                        
114507    [재료] 양배추 1/4통| 양파 1/2개| 편마늘 1숟가락| 식용유 적당히| 물 2...
114508    [재료] 박력분 200g| 버터 70g| 크림치즈 70g| 설탕 100g| 계란 1...
114509    [재료] 포항초 300g| 건다시마 작은크기4~5장| 다시멸치 1줌| 홍고추 1개|...
114510    [재료] 식빵 4장| 감자 2개| 양파 1/4개| 피망 1/4개| 계란 2개| 치즈...
114511    [재료] 배추잎 8~10장| 홍고추 1개| 계란 2개 [양념] 부침가루(메밀) 5큰...
Name: CKG_MTRL_CN, Length: 114512, dtype: object

소괄호 제거하는 함수

In [6]:
from collections import deque
def remove_small_brackets(word):
  start_q = deque()
  end_q = deque()
  index = set()
  for i in range(len(word)):
    if word[i] == '(':
      start_q.append(i)
  for i in range(len(word)):
    if word[i] == ')':
      end_q.append(i)
  
  while True:
    if len(start_q) == 0 or len(end_q) == 0:
      break
    index.add((start_q.popleft(), end_q.popleft()))
  index_list = list(index)
  if len(start_q) != 0 or len(end_q) != 0:
    return word
  else:
    for index in index_list:
      drop_word = word[index[0]:index[1]+1]
      word = word.replace(drop_word, '')
    return word

# 재료만 추출

대괄호를 제거하는 함수

In [7]:
from collections import deque
def remove_big_brackets(word):
  start_q = deque()
  end_q = deque()
  index = set()
  for i in range(len(word)):
    if word[i] == '[':
      start_q.append(i)
  for i in range(len(word)):
    if word[i] == ']':
      end_q.append(i)
  
  while True:
    if len(start_q) == 0 or len(end_q) == 0:
      break
    index.add((start_q.popleft(), end_q.popleft()))
  index_list = list(index)
  if len(start_q) != 0 or len(end_q) != 0:
    return word
  else:
    for index in index_list:
      drop_word = word[index[0]:index[1]+1]
      word = word.replace(drop_word, '')
    return word

전체 데이터를 돌면서 대괄호를 파악하여 '[양념]'을 포함하여 뒷 부분을 제거합니다.  
데이터의 모든 '[재료]'를 포함한 모든 대괄호를 제거합니다.

In [8]:
for i in tqdm(range(len(data))):
    temp = []
    for j in range(len(data[i])):
        if data[i][j] =='[':
            temp.append(j)
    end_index = len(data[i])
    if len(temp) > 1:
        end_index = temp[1]
    ingre = data[i][:end_index]
    ingre = remove_big_brackets(ingre).strip()
    data[i] = ingre

100%|████████████████████████████████████████████████████████████████████████| 114512/114512 [01:00<00:00, 1892.23it/s]


새로운 컬럼을 생성해줍니다.

In [9]:
df['only_ingred'] = None
df['only_ingred']

0         None
1         None
2         None
3         None
4         None
          ... 
114507    None
114508    None
114509    None
114510    None
114511    None
Name: only_ingred, Length: 114512, dtype: object

'|'기준으로 문장을 나누고, 나누어진 리스트에서 문자를 추출하여 양 옆 공백을 제거해준뒤  
문자에 포함되어 있는 소괄호를 제거해줍니다.

다시 공백을 기준으로 문자를 나눈 후에  
생성된 리스트의 길이가 3개가 이상이라면 마지막(단위부분)을 제외하고 추출합니다.  
만약 3개 이하라면 앞에 있는 데이터만 추출합니다.

In [10]:
#한쪽만 존재하는 소괄호를 제거하는 함수
def remove_one_bracket(word):
    if word.find('(') > -1:
        idx = word.find('(')
        return word[:idx]
    if word.find(')') > -1:
        idx = word.find(')')
        return word[:idx]
    return word

In [11]:
word = '소금(asd'
remove_one_bracket(word)

'소금'

In [12]:
for i in tqdm(range(len(data))):
    lst =[]
    for word in data[i].split('|'):
        word = word.strip()
        word = remove_small_brackets(word)
        word = remove_one_bracket(word)
        li = word.split()
        if li != []:
            if len(li) < 3:
                lst.append(li[0])
            else:
                string = ' '
                lst.append(string.join(li[:-1]))
    df['only_ingred'][i] = lst

  0%|                                                                                       | 0/114512 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Temp\ipykernel_24348\445727793.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['only_ingred'][i] = lst
100%|████████████████████████████████████████████████████████████████████████| 114512/114512 [00:32<00:00, 3559.31it/s]


In [13]:
df[['CKG_MTRL_CN','only_ingred']]

,CKG_MTRL_CN,only_ingred
0,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,"[어묵, 김밥용김, 당면, 양파, 당근, 깻잎, 튀김가루, 올리브유, 간장, 참기름]"
1,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,"[현미, 찹쌀, 호두, 물, 소금]"
2,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,"[북어포, 찹쌀가루]"
3,[재료] 파스타면 [양념] 토마토 1개| 토마토 페이스트 3T| 양파 1/2개| 다...,[파스타면]
4,[재료] 건표고버섯 9개| 오이 1/2개| 당근 1/2개| 양파 1/2개| 사과 1...,"[건표고버섯, 오이, 당근, 양파, 사과, 그외의, 과일]"
...,...,...
114507,[재료] 양배추 1/4통| 양파 1/2개| 편마늘 1숟가락| 식용유 적당히| 물 2...,"[양배추, 양파, 편마늘, 식용유, 물, 고체 카레]"
114508,[재료] 박력분 200g| 버터 70g| 크림치즈 70g| 설탕 100g| 계란 1...,"[박력분, 버터, 크림치즈, 설탕, 계란, 소금, 바닐라향]"
114509,[재료] 포항초 300g| 건다시마 작은크기4~5장| 다시멸치 1줌| 홍고추 1개|...,"[포항초, 건다시마, 다시멸치, 홍고추, 대파, 새송이버섯]"
114510,[재료] 식빵 4장| 감자 2개| 양파 1/4개| 피망 1/4개| 계란 2개| 치즈...,"[식빵, 감자, 양파, 피망, 계란, 치즈, 게맛살]"


In [14]:
empty_index_lst = []
for i in range(len(df['only_ingred'])):
    if df['only_ingred'][i] == []:
        empty_index_lst.append(i)

In [15]:
df[['RCP_SNO','CKG_MTRL_CN','only_ingred']].iloc[empty_index_lst,]

,RCP_SNO,CKG_MTRL_CN,only_ingred
21796,6829965,[재료] [냉면무]| 냉면무 800g | 소금 3작은술| 식초 6큰술 | 매실청(설...,[]
36007,6849719,[재료] [다쿠아즈] 흰자 95g(3개 분량)| 설탕 40g| 박력분 20g| 슈...,[]
38915,6853598,[] [탕종] 밀가루 15g| 물 75g [본반죽] 우리밀 60g| 강력분 175...,[]
40838,6856687,[재료 : 고등어 2마리| 대파 1뿌리 무 조림 재료 : 무1개| 물 1|200c...,[]
40896,6856769,[재료] [쿠키] 우리밀고소밀 160g| 비정제 설탕 40g| 계피가루 1ts| ...,[]
41369,6857405,[분량] [쿠키부분] 녹인버터 100g| 설탕 50g| 박력분 150g| 베이킹 ...,[]
41398,6857441,[분량] [쿠키반죽] 박력분 125g| 황치즈 분말 15g| 슈가파우더 50g| ...,[]
41447,6857503,[분량] [땅콩쿠키] 박력분 120g| 버터 40g| 땅콩버터 45g| 설탕 40...,[]
43900,6861605,[재료] [배우유빙수]| 우유 450g| 배 450g| 연유 60g| [연유배밀크잼...,[]
43940,6861670,[재료] [빵(8개 분량)] 강력분 200g| 물 95~115 g| 인스턴트 드라...,[]


In [16]:
empty_rcpno_lst = list(df[['RCP_SNO','CKG_MTRL_CN','only_ingred']].iloc[empty_index_lst,]['RCP_SNO'].values)

In [17]:
empty_rcpno_lst

[6829965,
 6849719,
 6853598,
 6856687,
 6856769,
 6857405,
 6857441,
 6857503,
 6861605,
 6861670,
 6861738,
 6862084,
 6862110,
 6885238,
 6920092,
 6921043,
 6924652,
 6924876,
 6929977,
 6933406,
 6936596,
 6944456,
 6947227,
 6948304,
 6948623,
 6951269,
 6951322]

In [18]:
ingre_list = []
for i in range(len(df['only_ingred'])):
    for word in df['only_ingred'][i]:
        ingre_list.append(word)

In [19]:
len(set(ingre_list))

47917

In [20]:
from collections import Counter
counter = Counter(ingre_list)
counter.most_common(10)

[('양파', 31241),
 ('대파', 19334),
 ('소금', 18672),
 ('물', 13413),
 ('당근', 13313),
 ('설탕', 12938),
 ('계란', 11724),
 ('청양고추', 10521),
 ('참기름', 8904),
 ('달걀', 8749)]

In [24]:
save_df = pd.DataFrame(df['only_ingred'])
save_df.to_csv('only_ingredient_list.csv', index=False)

# 양념 리스트 생성

In [228]:
data = df['CKG_MTRL_CN'].copy()
data

0         [재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...
1                  [재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간
2         [재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...
3         [재료] 파스타면 [양념] 토마토 1개| 토마토 페이스트 3T| 양파 1/2개| 다...
4         [재료] 건표고버섯 9개| 오이 1/2개| 당근 1/2개| 양파 1/2개| 사과 1...
                                ...                        
114507    [재료] 양배추 1/4통| 양파 1/2개| 편마늘 1숟가락| 식용유 적당히| 물 2...
114508    [재료] 박력분 200g| 버터 70g| 크림치즈 70g| 설탕 100g| 계란 1...
114509    [재료] 포항초 300g| 건다시마 작은크기4~5장| 다시멸치 1줌| 홍고추 1개|...
114510    [재료] 식빵 4장| 감자 2개| 양파 1/4개| 피망 1/4개| 계란 2개| 치즈...
114511    [재료] 배추잎 8~10장| 홍고추 1개| 계란 2개 [양념] 부침가루(메밀) 5큰...
Name: CKG_MTRL_CN, Length: 114512, dtype: object

In [229]:
data[0]

'[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개| 깻잎 6장| 튀김가루 1컵 | 올리브유 적당량| 간장 1T| 참기름 1T'

In [230]:
data[114511]

'[재료] 배추잎 8~10장| 홍고추 1개| 계란 2개 [양념] 부침가루(메밀) 5큰술| 소금 2꼬집| 해바라기유 5~6큰술'

In [231]:
for i in tqdm(range(len(data))):
    temp = []
    for j in range(len(data[i])):
        if data[i][j] =='[':
            temp.append(j)
    start_index = len(data[i])
    if len(temp) > 1:
        start_index = temp[1]
    ingre = data[i][start_index:]
    if ingre == '':
        ingre = '없음'
    ingre = remove_big_brackets(ingre).strip()
    data[i] = ingre

100%|██████████| 114512/114512 [00:31<00:00, 3655.12it/s]


In [167]:
for i in range(len(data[:100])):
    print(data[i])

없음
없음
간장 2T| 설탕 1T| 물 1T| 다진파 1T| 다진마늘 1T| 참기름 1T| 깨소금 1T| 후춧가루 약간
토마토 1개| 토마토 페이스트 3T| 양파 1/2개| 다진마늘 1T| 피망 1/2개| 올리브유 3T
녹말가루 2C| 물 1C| 계란 노른자 1개 [탕수 소| 설탕 1/2C| 식초 3T| 간장 1T| 녹말물 2T
통마늘 3개| 다진마늘 1T| 화이트와인또는우유| 올리브유또는버터| 마늘솔트또는소금  파인애플 2쪽| 마요네즈 2T| 머스터드 1T| 식초1/2T(또는 레몬즙) | 꿀 1T
없음
고추장| 다진마늘| 참기름| 맛소금| 마요네즈
고춧가루 2T| 고추장 1T| 물엿 1T| 간장 1T| 참기름 1T| 다진 마늘 1/2T| 깨 1/2T| 후추가루 약간
홀그레인머스터드| 허니머스터드| 다진양파
다진 마늘1/4t개| 깨소금| 맛소금| 참기름
없음
참기름 1/2t| 맛소금 약간 | 깨 약간
없음
머스터드| 피클| 마요네즈
없음
고추장 1T| 고춧가루 1T| 다진 마늘 1T| 참기름 1T| 설탕 1t| 간장 1t| 청주 1t| 후추 약간| 소금 약간
깨 3T| 꿀 5T
백세 카레가루 2T
없음
다진양파 1/2개| 빵가루| 밀가루| 계란| 소금| 후추
우유 270ml| 박력분 25g| 노른자 3개| 설탕 60g| 바닐라오일 약간| 버터 20g
버터 75그램| 설탕 75그램| 노른자 1개| 박력분 110그램| 바닐라오일 약간
없음
버터 1T| 육수 1C
고춧가루 약간
소금 약간
밀가루 1C| 물 1C| 혼다시 1T| 후추&소금 약간  돈가스소스| 마요네즈
간장 2T| 물엿 또는 설탕 1T| 참기름 1T| 다진 파 1T| 다진 마늘 1T | 소금 약간| 후추 약간
간장 70ml| 물 300ml| 설탕 1큰술| 물엿 or요리당 1큰술| 굴소스 1큰술| 고춧가루 2큰술| 마늘 1/2큰술| 후춧가루 1작은술
고춧가루 1컵(대략 8큰술)| 간장 1컵(8큰술)| 청량고추 3개| 통마늘 10개| 청주 3큰술| 설탕 3큰술| 생강가루 1/2큰술| 참기름 1+1/2큰술|

In [170]:
data[2].split('|')

['간장 2T',
 ' 설탕 1T',
 ' 물 1T',
 ' 다진파 1T',
 ' 다진마늘 1T',
 ' 참기름 1T',
 ' 깨소금 1T',
 ' 후춧가루 약간']

In [171]:
lst = []
for i in tqdm(range(len(data))):
    for word in data[i].split('|'):
        word = word.strip()
        if word != '없음':  
            word = remove_small_brackets(word)
            word = remove_one_bracket(word)
            li = word.split()
            if li != []:
                if len(li) < 3:
                    lst.append(li[0])
                else:
                    string = ' '
                    lst.append(string.join(li[:-1]))
    #df['only_ingred'][i] = lst

100%|██████████| 114512/114512 [00:01<00:00, 62495.09it/s]


In [178]:
from collections import Counter
temp = []
for word in lst:
    temp.append(word.replace(' ',''))
counter = Counter(temp)
counter.most_common(100)

[('다진마늘', 22298),
 ('설탕', 22241),
 ('소금', 18353),
 ('간장', 18256),
 ('참기름', 17595),
 ('고춧가루', 11985),
 ('후추', 11635),
 ('물', 10101),
 ('고추장', 9473),
 ('맛술', 7901),
 ('통깨', 7872),
 ('올리고당', 7534),
 ('식초', 6579),
 ('고추가루', 5536),
 ('마늘', 4773),
 ('굴소스', 4195),
 ('국간장', 4003),
 ('깨소금', 3986),
 ('진간장', 3981),
 ('물엿', 3372),
 ('후춧가루', 3361),
 ('들기름', 3093),
 ('된장', 3080),
 ('양파', 3079),
 ('매실청', 3058),
 ('마요네즈', 2867),
 ('매실액', 2866),
 ('청주', 2832),
 ('식용유', 2500),
 ('깨', 2465),
 ('대파', 2378),
 ('버터', 2345),
 ('꿀', 2277),
 ('멸치액젓', 2030),
 ('다진파', 1857),
 ('레몬즙', 1731),
 ('새우젓', 1728),
 ('올리브유', 1590),
 ('청양고추', 1523),
 ('액젓', 1450),
 ('생강가루', 1431),
 ('미림', 1402),
 ('생강', 1380),
 ('파', 1346),
 ('우유', 1342),
 ('후추가루', 1291),
 ('케찹', 1241),
 ('다진생강', 1214),
 ('다시마', 1211),
 ('참치액', 1198),
 ('올리브오일', 1019),
 ('양조간장', 966),
 ('간마늘', 941),
 ('생크림', 922),
 ('생강즙', 810),
 ('파슬리가루', 810),
 ('생강술', 801),
 ('밀가루', 784),
 ('달걀', 776),
 ('케첩', 775),
 ('들깨가루', 775),
 ('계란', 759),
 ('까나리액젓', 726),
 ('맛간장